In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

In [ ]:
data = pd.read_csv('/content/OOR.V.csv')
data = data[["Date","Close"]]

# Converting the 'Date' column to a date type
data['Date'] = pd.to_datetime(data['Date'])
# Create a date range from the minimum to the maximum date in the initial dataset
date_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max())
print(data)
# Complete dataset with all dates
full_df = pd.DataFrame({'Date': date_range})
# Merging the original dataset with the full dataset to add missing dates
data = pd.merge(full_df, data, on='Date', how='left')
# Filling 'Close' values ​​to None for new dates
data['Close'] = np.where(data['Close'].isna(), None, data['Close'])
data['Close'] = data['Close'].replace(to_replace=None, method='ffill')

# Leave three months for the test
data_2022_2023 = data[(data['Date'] >= '2022-07-22') & (data['Date'] <= '2022-10-22')]
# Change the date format
data_2022_2023.loc[:, 'Date'] = pd.to_datetime(data_2022_2023.loc[:,'Date'], format="%Y/%m/%d")
# Leave 7 years left for model training
data = data[(data['Date'] >= '2015-07-21') & (data['Date'] < '2022-07-22')]
data = data.rename(columns = {"Date":"ds","Close":"y"})
print(data)

In [ ]:
m = Prophet(daily_seasonality = True)
m.fit(data)
# Predict for 90 days - three months
future = m.make_future_dataframe(periods=93)
prediction = m.predict(future)

# Building a plot
m.plot(prediction)
plt.title("Prediction of the Amazon Stock Price using the Prophet")
plt.xlabel("Date")
plt.ylabel("Close Stock Price")
plt.show()

In [ ]:
# Run the cycle for 47 iterations of 2 days - three months
for i in range(47):
    # Train the model for seven years and predict the next two days
    m = Prophet(daily_seasonality = True)
    m.fit(data)
    future = m.make_future_dataframe(periods=2)
    prediction_2days = m.predict(future)
    # In prediction_df leave only those days that was just predicted
    prediction_df = prediction_2days[prediction_2days['ds'] > data["ds"].max()]
    # Add them to data df
    row = pd.DataFrame({"ds": prediction_df['ds'],
                        "y": prediction_df['trend']})
    data = pd.concat([data, row])
    # Change the date format again
    data.loc[:, 'ds'] = pd.to_datetime(data.loc[:,'ds'], format="%Y/%m/%d")
print(data)

In [ ]:
formatter = mdates.DateFormatter("%Y")
locator = mdates.YearLocator()
fig = plt.figure(figsize=(7, 5))
ax = plt.gca()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(locator)
plt.scatter(data_2022_2023['Date'], data_2022_2023['Close'], s=10)
plt.tight_layout()
plt.show()


In [ ]:
prediction_df = prediction[(prediction['ds'] >= '2022-07-22') & (prediction['ds'] <= '2022-10-22')]
prediction_trend = prediction_df['trend'].values
actual = data_2022_2023['Close'].values
mape = mean_absolute_percentage_error(actual, prediction_trend)
r2 = r2_score(actual, prediction_trend)
MSE = np.square(np.subtract(actual, prediction_trend)).mean()
RMSE = MSE**(0.5)
print('MAPE', mape, '\nR^2', r2, '\nRMSE', RMSE)

In [ ]:
prediction_trend_2 = data[(data['ds'] >= '2022-07-22') & (data['ds'] <= '2022-10-22')]
prediction_trend_2 = prediction_trend_2['y'].values
actual2 = data_2022_2023['Close'].values
mape2 = mean_absolute_percentage_error(actual2, prediction_trend_2)
r2_2 = r2_score(actual2, prediction_trend_2)
MSE2 = np.square(np.subtract(actual2, prediction_trend_2)).mean()
RMSE2 = MSE2**(0.5)
print('MAPE2', mape2, '\nR^2', r2_2, '\nRMSE', RMSE2)